In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
  #original
    def __init__(self):
        #our var declaration
        super(Net, self).__init__()                     #Input - Output - RF
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #28 - 28 - 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)    #28 - 28 - 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                 #28 - 14 - 10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)   #14 - 14 - 12x12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)  #14 - 14 - 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                 #14 -  7 - 28x28
        self.conv5 = nn.Conv2d(256, 512, 3)             #7  -  5 - 30x30
        self.conv6 = nn.Conv2d(512, 1024, 3)            #5 -   3 - 32x32
        self.conv7 = nn.Conv2d(1024, 10, 3)             #3 -   1 - 34x34

    def forward(self, x):
        #self is containing all variables, compulsory to pass
        #x is input image
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))   #conv1 -> ReLU -> conv2 -> ReLU -> max pool
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))   #conv3 -> ReLU -> conv4 -> ReLU -> max pool
        x = F.relu(self.conv6(F.relu(self.conv5(x))))               #conv5 -> ReLU -> conv6 -> ReLU
        #x = F.relu(self.conv7(x))                                  #Dont need to perform ReLU on last layer as its output would be subject to softmax, cannot apply non-linearity twice.
        x = (self.conv7(x))                                         #conv7 -> output
        x = x.view(-1, 10)
        return F.log_softmax(x)       #log of softmax

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))          #In future check why this normalization works 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)   #send data to device/GPU
        optimizer.zero_grad()                               #reset to zero gradient
        output = model(data)                                #apply model
        loss = F.nll_loss(output, target)                   #get loss
        loss.backward()                                     #propagate loss backward
        optimizer.step()                                    
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),   #adjusted the decimal place to print finer accuracy
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0701, Accuracy: 9764/10000 (97.64%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0342, Accuracy: 9880/10000 (98.80%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0257, Accuracy: 9920/10000 (99.20%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0215, Accuracy: 9930/10000 (99.30%)



loss=0.004171313252300024 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.53it/s]



Test set: Average loss: 0.0282, Accuracy: 9926/10000 (99.26%)



In [7]:
class Net7(nn.Module):
  #original
    def __init__(self):
        super(Net7, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = (self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [8]:
model = Net7().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 32, 28, 28]           9,248
         MaxPool2d-3           [-1, 32, 14, 14]               0
            Conv2d-4          [-1, 256, 14, 14]          73,984
            Conv2d-5          [-1, 256, 14, 14]         590,080
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,665,578
Trainable params: 6,665,578
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.46
Params size (MB): 25.43
Estimated Total Size (MB): 26.89
-------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0656, Accuracy: 9774/10000 (97.74%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0446, Accuracy: 9855/10000 (98.55%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0362, Accuracy: 9902/10000 (99.02%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0304, Accuracy: 9916/10000 (99.16%)



loss=0.00023597425024490803 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.74it/s]



Test set: Average loss: 0.0315, Accuracy: 9913/10000 (99.13%)



In [ ]:
class Net8(nn.Module):
  #original
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 256, 3, padding=1)
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = (self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [ ]:
model = Net8().to(device)
summary(model, input_size=(1, 28, 28))

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

total_epochs = 10
for epoch in range(1, total_epochs+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

In [11]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        #transforms.Normalize((0.1307,), (0.3081,))          #In future check why this normalization works 
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        #transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:
#loader = DataLoader(train_loader, batch_size=len(train_loader), num_workers=1)
data = next(iter(test_loader))
data[0].mean(), data[0].std()

(tensor(0.1290), tensor(0.3065))

In [18]:
#https://deeplizard.com/learn/video/lu7TCu7HeYc
train_set = datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        #transforms.Normalize((0.1307,), (0.3081,))          #In future check why this normalization works 
                    ]))

num_of_pixels = len(train_set) * 28 * 28
total_sum = 0
for batch in train_loader: total_sum += batch[0].sum()
mean = total_sum / num_of_pixels
sum_of_squared_error = 0
for batch in train_loader: 
    sum_of_squared_error += ((batch[0] - mean).pow(2)).sum()
std = torch.sqrt(sum_of_squared_error / num_of_pixels)
mean, std

(tensor(0.1307), tensor(0.3081))